# Équipe 7
## Notebook de Jean-Christophe

In [1]:
import numpy as np
import pandas as pd
import datetime

import json

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash

import plotly.graph_objects as go
import plotly.express as px

## Treemap 

### Functions

In [2]:
def clean_year_column(df):
    temp = df.dropna(subset=['annee'])
    temp = temp.astype({'annee': 'int'})
    temp = temp[temp['annee']<=datetime.datetime.now().year]
    return temp

In [3]:
def clean_genre_column(df):
    temp = df.dropna(subset=['genre'])
    temp = temp.astype({'genre': 'str'})
    temp['genre'] = temp['genre'].apply(str.capitalize)
    
    return temp

In [4]:
def clean_title_column(df):
    temp = df.dropna(subset=['titre'])
    temp = temp.astype({'titre': 'str'})
    temp['titre'] = temp['titre'].apply(str.capitalize)
    
    return temp

In [5]:
def load_data_for_treemap():
    relative_path = "../Src/Assets/Data/"
    file_name = "_film_genres.csv"
    full_path = relative_path + file_name
    df_film_genres = pd.read_csv(
        filepath_or_buffer=full_path,
        sep=',',
        header=0,
        index_col=None,
        parse_dates=True,
    )

    relative_path = "../Src/Assets/Data/"
    file_name = "_film_pays.csv"
    full_path = relative_path + file_name
    df_film_pays = pd.read_csv(
        filepath_or_buffer=full_path,
        sep=',',
        header=0,
        index_col=None,
        parse_dates=True,
    )

    relative_path = "../Src/Assets/Data/"
    file_name = "_film_langue.csv"
    full_path = relative_path + file_name
    df_film_langues = pd.read_csv(
        filepath_or_buffer=full_path,
        sep=',',
        header=0,
        index_col=None,
        parse_dates=True,
    )

    relative_path = "../Src/Assets/Data/"
    file_name = "pays_vsParContinent_vsFrancais.csv"
    full_path = relative_path + file_name
    df_pays = pd.read_csv(
        filepath_or_buffer=full_path,
        sep=',',
        header=0,
        index_col=None,
        parse_dates=True,
    )

    temp = pd.merge(
        left=df_film_pays,
        right=df_pays,
        how="left",
        left_on="pays",
        right_on="pays",
        sort=True,
        suffixes=("", ""),
    )

    temp = pd.merge(
        left=temp,
        right=df_film_langues,
        how="left",
        left_on="filmoId",
        right_on="filmoId",
        sort=True,
        suffixes=("", ""),
    )

    temp = pd.merge(
        left=temp,
        right=df_film_genres,
        how="left",
        left_on="filmoId",
        right_on="filmoId",
        sort=True,
        suffixes=("", ""),
    )

    temp = temp[["anneeSortie", "continent", "pays", "capitale", "langue", "filmoId", "titreOriginal", "genres_categorized"]]
    treemap_df = temp.rename(columns={"anneeSortie": "annee", "filmoId": "id", "titreOriginal": "titre", "genres_categorized": "genre"})

    return treemap_df

In [6]:
def get_min_and_max_year(df):
    year_max = df['annee'].max()
    year_min = df['annee'].min()
    
    return year_min, year_max

In [7]:
def get_list_of_genres(df):
    genres = list(set(df['genre'].values))
    
    return genres

In [8]:
def add_constant_col(df, col_name, const_name):
    temp_df = df.copy()
    temp_df[col_name] = const_name
    
    return temp_df

In [9]:
def query_data_for_treemap(df, genre, from_year, to_year):
    
    # Extract data corresponding to the right genre
    if genre == "All":
        temp_df = df
    else:
        query_str = "genre=='" + genre + "'"
        temp_df = df.query(query_str)
    
    # Extract data corresponding to the right years
    query_str = "annee>=" + str(from_year)
    temp_df = temp_df.query(query_str)
    query_str = "annee<=" + str(to_year)
    temp_df = temp_df.query(query_str)
        
    # Aggregate agnostic to years
    temp_df = (temp_df
        .groupby(['planete', 'continent', 'pays'])
        .agg(nombreDeFilms=pd.NamedAgg(column='genre', aggfunc='count'))
        .reset_index()
    )
    return temp_df

In [10]:
def create_treemap_dropdown_menu(genres):
    # Create options
    options=[{'label': "All", 'value': "All"}]
    for genre in genres:
        option = {'label': genre, 'value': genre}
        options.append(option)

    # Create menu
    ddm = dcc.Dropdown(
            id='treemap-dropdown-menu',
            options=options,
            value='All',
        )
    
    return ddm

In [11]:
def create_treemap_title(year_min_displayed, year_max_displayed, genre_displayed):
    
    if genre_displayed=="All":
        title_str = (
            'Nombre de films de tous genres produits entre les années ' + 
            str(year_min_displayed) + 
            ' et ' + 
            str(year_max_displayed) + 
            '.'
        )
    else: 
        title_str = (
            'Nombre de films de genre ' + 
            genre_displayed + 
            ' produit entre les années ' + 
            str(year_min_displayed) + 
            ' et ' + 
            str(year_max_displayed) + 
            '.'
        )

    title = html.Header( 
        children=[
            html.H2(id='treemap-title', children=title_str)
        ]
    )
    
    return title

In [12]:
def create_treemap_range_slider(year_min, year_max):
    
    # Create list of years
    first_decade = 10 - (year_min%10) + year_min
    last_decade = year_max - (year_max%10)
    years = np.arange(start=first_decade, stop=last_decade+1, step=10)

    rs = dcc.RangeSlider(
        id='treemap-range-slider',
        min=year_min,
        max=year_max,
        value=[year_min, year_max],
        pushable=False,
        allowCross=True,
        dots=False,
        updatemode='mouseup',
        marks={str(year): str(year) for year in years},
    )    
    
    return rs

In [13]:
def init_treemap_fig(df, year_min, year_max):
    # Get data
    temp_df = query_data_for_treemap(
        df=df, 
        genre='All', 
        from_year=year_min, 
        to_year=year_max
    )

    # Create a figure with px
    fig = px.treemap(
        temp_df, 
        path=['planete', 'continent', 'pays'], 
        values='nombreDeFilms',
    )

    return fig

In [14]:
def callback_treemap(df, slider_min_year, slider_max_year, ddm_genre):
    
    # Create new title
    treemap_title = create_treemap_title(
        year_min_displayed=slider_min_year, 
        year_max_displayed=slider_max_year, 
        genre_displayed=ddm_genre
    )
    
    # Get data
    temp_df = query_data_for_treemap(
        df=df, 
        genre=ddm_genre, 
        from_year=slider_min_year, 
        to_year=slider_max_year
    )

    # Create a figure with px
    treemap_fig = px.treemap(
        temp_df, 
        path=['planete', 'continent', 'pays'], 
        values='nombreDeFilms',
    )

    return treemap_title, treemap_fig

In [15]:
def create_treemap_graph(treemap_fig):
    treemap_graph = dcc.Graph(
        figure=treemap_fig, 
        id='treemap-graph',
        config=dict(
            showTips=False,
            showAxisDragHandles=False,
            displayModeBar=False
        )
    )

    return treemap_graph

In [16]:
def create_treemap(treemap_title, treemap_rs, treemap_ddm, treemap_graph):
    
    treemap = html.Div([
        treemap_title,
        treemap_ddm,
        treemap_graph,
        treemap_rs
    ])

    return treemap

## Table
### Test - With real data

In [25]:
# Create list of years
max_top = 10
top_n = np.arange(start=1, stop=max_top+1, step=1)

rs = dcc.RangeSlider(
    id='bar-range-slider',
    min=1,
    max=max_top,
    value=[5],
    pushable=False,
    allowCross=False,
    dots=False,
    updatemode='mouseup',
    marks={str(i): str(i) for i in top_n},
) 

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Notes sur la librairie Dash de Plotly"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    rs,
    html.Div(id='output-range-slider')
])
@app.callback(
    dash.dependencies.Output('output-range-slider', 'children'),
    [dash.dependencies.Input('range-slider', 'value')])
def update_output(value):
    return 'You have selected "{}"'.format(value)

app.run_server(mode='inline', port=8030)

In [26]:
# Create a table 
table = html.Div(
    style = {
        'width': '100%', 
        'display': 'flex', 
        'alignItems': 'center', 
        'justifyContent': 'center', 
        'flexDirection' : 'column'
    }, 
    children=[
        html.H4('Hover over markers for more information', id='country'),
        html.Table(
            children=[
                html.Thead(
                    html.Tr(
                        children=[
                            html.Th('Langue'), 
                            html.Th('Nombre de production'), 
                            html.Th('Proportion')
                        ]
                    )
                ),
                html.Tbody(
                    children=[
                        html.Tr(
                            children=[
                                html.Td('2000'), 
                                html.Td("id='2000-gdp'"), 
                                html.Td("id='2000-co2'")
                            ]
                        ),
                        html.Tr(
                            children=[
                                html.Td('2015'), 
                                html.Td("id='2015-gdp'"),
                                html.Td("id='2015-co2'")
                            ]
                        ),
                    ]
                )
            ]
        )
    ]
)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Notes sur la librairie Dash de Plotly"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    table
])

app.run_server(mode='inline', port=8030)

In [33]:
# Create a table 
table = html.Div(
    style = {
        'width': '100%', 
        'display': 'flex', 
        'alignItems': 'center', 
        'justifyContent': 'center', 
        'flexDirection' : 'column'
    }, 
    children=[
        html.H4('Hover over markers for more information', id='country'),
        html.Table(children=[
            html.Tr(children=[
                html.Td('1'),
                html.Td(
                    children=html.Table(
                        children=[
                            html.Thead(
                                html.Tr(
                                    children=[
                                        html.Th('Langue'), 
                                        html.Th('Nombre de production'), 
                                        html.Th('Proportion')
                                    ]
                                )
                            ),
                            html.Tbody(
                                children=[
                                    html.Tr(
                                        children=[
                                            html.Td('2000'), 
                                            html.Td("id='2000-gdp'"), 
                                            html.Td("id='2000-co2'")
                                        ]
                                    ),
                                    html.Tr(
                                        children=[
                                            html.Td('2015'), 
                                            html.Td("id='2015-gdp'"),
                                            html.Td("id='2015-co2'")
                                        ]
                                    )
                                ]
                            )
                        ]
                    ), rowSpan='2', style={'border-style':'solid','border-width':'1px'},
                ),
                html.Td(
                    children=html.Table(
                        children=[
                            html.Thead(
                                html.Tr(
                                    children=[
                                        html.Th('Langue'), 
                                        html.Th('Nombre de production'), 
                                        html.Th('Proportion')
                                    ]
                                )
                            ),
                            html.Tbody(
                                children=[
                                    html.Tr(
                                        children=[
                                            html.Td('2000'), 
                                            html.Td("id='2000-gdp'"), 
                                            html.Td("id='2000-co2'")
                                        ]
                                    ),
                                    html.Tr(
                                        children=[
                                            html.Td('2015'), 
                                            html.Td("id='2015-gdp'"),
                                            html.Td("id='2015-co2'")
                                        ]
                                    )
                                ]
                            )
                        ]
                    ), rowSpan='2', style={'border-style':'solid','border-width':'1px'},
                )]
            ),
            html.Tr(
                html.Td(children=['2'])
           )]
        )
    ]
)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Notes sur la librairie Dash de Plotly"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    table
])

app.run_server(mode='inline', port=8030)

In [35]:
# Create a table 
table = html.Div(
    style = {
        'width': '100%', 
        'display': 'flex', 
        'alignItems': 'center', 
        'justifyContent': 'center', 
        'flexDirection' : 'column'
    }, 
    children=[
        html.H4('Hover over markers for more information', id='country'),
        html.Table(children=[
            html.Tr(children=[
                html.Td(
                    children=html.Table(
                        children=[
                            html.Thead(
                                html.Tr(
                                    children=[
                                        html.Th('Langue'), 
                                        html.Th('Nombre de production'), 
                                        html.Th('Proportion')
                                    ]
                                )
                            ),
                            html.Tbody(
                                children=[
                                    html.Tr(
                                        children=[
                                            html.Td('2000'), 
                                            html.Td("id='2000-gdp'"), 
                                            html.Td("id='2000-co2'")
                                        ]
                                    ),
                                    html.Tr(
                                        children=[
                                            html.Td('2015'), 
                                            html.Td("id='2015-gdp'"),
                                            html.Td("id='2015-co2'")
                                        ]
                                    )
                                ]
                            ),
                            html.Tfoot()
                        ]
                    ), rowSpan='2', colSpan='2', style={'border-style':'solid','border-width':'1px'},
                ),
                html.Td(
                    children=html.Table(
                        children=[
                            html.Thead(
                                html.Tr(
                                    children=[
                                        html.Th('Langue'), 
                                        html.Th('Nombre de production'), 
                                        html.Th('Proportion')
                                    ]
                                )
                            ),
                            html.Tbody(
                                children=[
                                    html.Tr(
                                        children=[
                                            html.Td('2000'), 
                                            html.Td("id='2000-gdp'"), 
                                            html.Td("id='2000-co2'")
                                        ]
                                    ),
                                    html.Tr(
                                        children=[
                                            html.Td('2015'), 
                                            html.Td("id='2015-gdp'"),
                                            html.Td("id='2015-co2'")
                                        ]
                                    )
                                ]
                            )
                        ]
                    ), rowSpan='1', colSpan='1', style={'border-style':'solid','border-width':'1px'},
                )]
            ),
            html.Tr(
                html.Td(
                    children=html.Table(
                        children=[
                            html.Thead(
                                html.Tr(
                                    children=[
                                        html.Th('Langue'), 
                                        html.Th('Nombre de production'), 
                                        html.Th('Proportion')
                                    ]
                                )
                            ),
                            html.Tbody(
                                children=[
                                    html.Tr(
                                        children=[
                                            html.Td('2000'), 
                                            html.Td("id='2000-gdp'"), 
                                            html.Td("id='2000-co2'")
                                        ]
                                    ),
                                    html.Tr(
                                        children=[
                                            html.Td('2015'), 
                                            html.Td("id='2015-gdp'"),
                                            html.Td("id='2015-co2'")
                                        ]
                                    )
                                ]
                            )
                        ]
                    ), rowSpan='1', colSpan='1', style={'border-style':'solid','border-width':'1px'},
                )
           )]
        )
    ]
)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Notes sur la librairie Dash de Plotly"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    table
])

app.run_server(mode='inline', port=8030)

### Functions

In [18]:
def round_decimals(df, n):
    # TODO : Round the dataframe
    temp_df = df.round(decimals=n)
    
    return temp_df

In [19]:
def get_list_of_languages(df):
    languages = list(set(df['langue'].values))
    
    return languages

In [20]:
def drop_duplicated_movies(df):
    subset=['annee', 'continent', 'pays', 'capitale', 'langue', 'id', 'titre']
    temp = df.drop_duplicates(subset=subset, keep='first')
    
    return temp

In [21]:
def clean_language_column(df):
    temp = df.copy()
    temp['langue'] = (temp['langue']
        .apply(lambda x: str.capitalize(x) if isinstance(x, str) else x)
    )
    
    return temp

In [22]:
def query_data_for_table(df, genre, from_year, to_year, top_n):
    
    def replace_string(x):
        if x in top_n_languages:
            return x
        else:
             return 'autres' 
         
        
    # Extract data corresponding to the right genre
    if genre == "All":
        temp_df = df
    else:
        query_str = "genre=='" + genre + "'"
        temp_df = df.query(query_str)
    
    # Extract data corresponding to the right years
    query_str = "annee>=" + str(from_year)
    temp_df = temp_df.query(query_str)
    query_str = "annee<=" + str(to_year)
    temp_df = temp_df.query(query_str)
    
    # get count of movies for each language
    top_n_df = (temp_df
        .groupby('langue')
        .agg(nombreDeFilms=pd.NamedAgg(column='annee', aggfunc='count'))
        .reset_index()
        .sort_values(['nombreDeFilms'], ascending=[False])
    )
    
    # sort to have descending order
    #top_n_df = (top_n_df
    #    .reset_index()
    #    .sort_values(['nombreDeFilms'], ascending=[False])
    #)
    
    # handles cases where there is more than n languages
    if len(top_n_df) > top_n:
        # create a list with top n languages
        top_n_languages = top_n_df.head(top_n)['langue'].tolist()

        # replace non top n languages by "autres"
        top_n_df['langue'] = top_n_df['langue'].apply(replace_string)

        # aggregate to have top n + autres languages
        top_n_df = (top_n_df
            .groupby(['langue'])
            .agg(nombreDeFilms=pd.NamedAgg(column='nombreDeFilms', aggfunc='sum'))
            .reset_index()
            .sort_values(['nombreDeFilms'], ascending=[False])
        )
    # else: no need to do anything
    
    # get the percentage of each language
    n_films = top_n_df['nombreDeFilms'].sum()
    top_n_df['pourcentageDeFilms'] = (top_n_df['nombreDeFilms']
        .apply(lambda x: 100 * x / n_films)
    )
    
    top_n_df = round_decimals(df=top_n_df, n=2)
    
    return top_n_df

In [23]:
def query_data_for_table(df, genre, from_year, to_year, top_n):
    
    def replace_string(x):
        if x in top_n_languages:
            return x
        else:
             return 'autres' 
         
        
    # Extract data corresponding to the right genre
    if genre == "All":
        temp_df = df
    else:
        query_str = "genre=='" + genre + "'"
        temp_df = df.query(query_str)
    
    # Extract data corresponding to the right years
    query_str = "annee>=" + str(from_year)
    temp_df = temp_df.query(query_str)
    query_str = "annee<=" + str(to_year)
    temp_df = temp_df.query(query_str)
    
    # Extract oldest movie from that period
    oldest_date = temp_df['annee'].min()
    query_str = "annee==" + str(oldest_date)
    oldest_df = temp_df.query(query_str)
    
    # Extract latest movie from that period
    latest_date = temp_df['annee'].max()
    query_str = "annee==" + str(latest_date)
    latest_df = temp_df.query(query_str)
    
    # get count of movies for each language
    top_n_df = (temp_df
        .groupby('langue')
        .agg(nombreDeFilms=pd.NamedAgg(column='annee', aggfunc='count'))
        .reset_index()
        .sort_values(['nombreDeFilms'], ascending=[False])
    )
    
    # sort to have descending order
    #top_n_df = (top_n_df
    #    .reset_index()
    #    .sort_values(['nombreDeFilms'], ascending=[False])
    #)
    
    # handles cases where there is more than n languages
    if len(top_n_df) > top_n:
        # create a list with top n languages
        top_n_languages = top_n_df.head(top_n)['langue'].tolist()

        # replace non top n languages by "autres"
        top_n_df['langue'] = top_n_df['langue'].apply(replace_string)

        # aggregate to have top n + autres languages
        top_n_df = (top_n_df
            .groupby(['langue'])
            .agg(nombreDeFilms=pd.NamedAgg(column='nombreDeFilms', aggfunc='sum'))
            .reset_index()
            .sort_values(['nombreDeFilms'], ascending=[False])
        )
    # else: no need to do anything
    
    # get the percentage of each language
    n_films = top_n_df['nombreDeFilms'].sum()
    top_n_df['pourcentageDeFilms'] = (top_n_df['nombreDeFilms']
        .apply(lambda x: 100 * x / n_films)
    )
    
    top_n_df = round_decimals(df=top_n_df, n=2)
    
    return top_n_df, oldest_df, latest_df

In [25]:
def create_table_title(year_min_displayed, year_max_displayed, genre_displayed, top_n):
    
    if genre_displayed=="All":
        title_str = (
            'Top ' +
            str(top_n) +
            ': Distribution des langues dans les films de tous genres produits entre les années ' + 
            str(year_min_displayed) + 
            ' et ' + 
            str(year_max_displayed) + 
            '.'
        )
    else: 
        title_str = (
            'Top ' +
            str(top_n) +
            ': Distribution des langues dans les films de genre ' + 
            genre_displayed + 
            ' produit entre les années ' + 
            str(year_min_displayed) + 
            ' et ' + 
            str(year_max_displayed) + 
            '.'
        )

    title = html.H4(
        id='table-title', 
        children=title_str
    )
    
    return title

In [38]:
## Create slider
def create_table_slider(initial_value):
    
    # Create list of years
    max_top = 10
    top_n = np.arange(start=1, stop=max_top+1, step=1)

    slider = dcc.RangeSlider(
        id='table-slider',
        min=1,
        max=max_top,
        value=[initial_value],
        pushable=False,
        allowCross=False,
        dots=False,
        updatemode='mouseup',
        marks={str(i): str(i) for i in top_n},
    )    
    
    return slider

In [27]:
def create_table_body(df, genre, from_year, to_year, top_n):
    
    # Get the top 5 languages df
    top_n_df = query_data_for_table(
        df=df, 
        genre=genre,
        from_year=from_year,
        to_year=to_year, 
        top_n=top_n
    )
    
    # Get data to display in table body
    languages = top_n_df['langue'].values.tolist()
    count = top_n_df['nombreDeFilms'].values.tolist()
    percentage = top_n_df['pourcentageDeFilms'].values.tolist()
    
    # Create blanc table body
    table_body=[]
      
    # Fill the table body
    for idx, language in enumerate(languages):
        table_row = html.Tr(
            children=[
                html.Td(language), 
                html.Td(str(count[idx])), 
                html.Td(str(percentage[idx])+"%")
            ]
        )
        
        table_body.append(table_row)
    
    # Create and add slider to last row of table
    #table_slider = create_table_slider(initial_value=top_n)
    #table_row = html.Tr(
    #    children=[
    #        html.Td(
    #            style={'colspan': '3'}, 
    #            children=table_slider
    #        )
    #    ]
    #)
    #table_body.append(table_row)
    
    return table_body

In [44]:
def create_table_body(df, genre, from_year, to_year, top_n):
    
    # Get the top 5 languages df
    top_n_df, oldest_df, latest_df = query_data_for_table(
        df=df, 
        genre=genre,
        from_year=from_year,
        to_year=to_year, 
        top_n=top_n
    )
    
    # Get data to display in table body for top n movies
    languages = top_n_df['langue'].values.tolist()
    count = top_n_df['nombreDeFilms'].values.tolist()
    percentage = top_n_df['pourcentageDeFilms'].values.tolist()
    
    # Create blanc table body
    table_body_top_n=[]
      
    # Fill the table body
    for idx, language in enumerate(languages):
        table_row = html.Tr(
            children=[
                html.Td(language), 
                html.Td(str(count[idx])), 
                html.Td(str(percentage[idx])+"%")
            ]
        )
        
        table_body_top_n.append(table_row)
    
    # Get data to display in table body
    titles = oldest_df['titre'].values.tolist()
    years = oldest_df['annee'].values.tolist()
    languages = oldest_df['langue'].values.tolist()
    
    # Create blanc table body
    table_body_oldest=[]
      
    # Fill the table body
    for idx, title in enumerate(titles):
        table_row = html.Tr(
            children=[
                html.Td(title), 
                html.Td(str(years[idx])), 
                html.Td(languages[idx])
            ]
        )
        
        table_body_oldest.append(table_row)
    
    # Get data to display in table body
    titles = latest_df['titre'].values.tolist()
    years = latest_df['annee'].values.tolist()
    languages = latest_df['langue'].values.tolist()
    
    # Create blanc table body
    table_body_latest=[]
      
    # Fill the table body
    for idx, title in enumerate(titles):
        table_row = html.Tr(
            children=[
                html.Td(title), 
                html.Td(str(years[idx])), 
                html.Td(languages[idx])
            ]
        )
        
        table_body_latest.append(table_row)
    
    return table_body_top_n, table_body_oldest, table_body_latest

In [29]:
def callback_table(df, slider_min_year, slider_max_year, ddm_genre, top_n):
    
    # Create new title
    table_title = create_table_title(
        year_min_displayed=slider_min_year, 
        year_max_displayed=slider_max_year, 
        genre_displayed=ddm_genre,
        top_n=top_n
    )
    
    # Create a figure with px
    table_body = create_table_body(
        df=df,
        genre=ddm_genre,
        from_year=slider_min_year, 
        to_year=slider_max_year, 
        top_n=top_n,
    )

    return table_title, table_body

In [30]:
def callback_table(df, slider_min_year, slider_max_year, ddm_genre, top_n):
    
    # Create new title
    table_title = create_table_title(
        year_min_displayed=slider_min_year, 
        year_max_displayed=slider_max_year, 
        genre_displayed=ddm_genre,
        top_n=top_n
    )
    
    # Create a figure with px
    table_body_top_n, table_body_oldest, table_body_latest = create_table_body(
        df=df,
        genre=ddm_genre,
        from_year=slider_min_year, 
        to_year=slider_max_year, 
        top_n=top_n,
    )

    return table_title, table_body_top_n, table_body_oldest, table_body_latest

In [31]:
def create_table(table_title, table_body, table_slider):
    
    table1 = html.Div(
        children=[
            table_title,
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    'width': '100%', 
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Langue'), 
                                html.Th('Nombre de production'), 
                                html.Th('Proportion')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'table-body',
                        children=table_body,
                    )
                ],
            ),
            table_slider,
        ],  
    )
    
    table2 = html.Div(
        children=[
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    'width': '100%', 
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Langue'), 
                                html.Th('Nombre de production'), 
                                html.Th('Proportion')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'tabledeux',
                        children=table_body,
                    )
                ],
            ),
        ],  
    )
        
    table3 = html.Div(
        children=[
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Langue'), 
                                html.Th('Nombre de production'), 
                                html.Th('Proportion')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'tabletrois',
                        children=table_body,
                    )
                ],
            ),
        ],  
    )
    
    table = html.Table(
        children=[
            html.Tr(
                children=[
                    html.Td(
                        children=[table1],
                        rowSpan='2',
                        colSpan='2',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    ),
                    html.Td(
                        children=[table2],
                        rowSpan='1',
                        colSpan='1',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    )
                ]
            ),
            html.Tr(
                children=[
                    html.Td(
                        children=[table3],
                        rowSpan='1',
                        colSpan='1',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    )
                ]
            )
        ]
    )
                     
    return table

In [32]:
def create_table(table_title, table_body_top_n, table_body_oldest, table_body_latest, table_slider):
    
    table_top_n = html.Div(
        children=[
            table_title,
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    'width': '100%', 
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Langue'), 
                                html.Th('Nombre de production'), 
                                html.Th('Proportion')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'table-body-top-n',
                        children=table_body_top_n,
                    )
                ],
            ),
            table_slider,
        ],  
    )
    
    table_oldest = html.Div(
        children=[
            html.H5("Les plus anciennes productions"),
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    'width': '100%', 
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Titre'), 
                                html.Th('Année'), 
                                html.Th('Langue')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'table-body-oldest',
                        children=table_body_oldest,
                    )
                ],
            ),
        ],  
    )
        
    table_latest = html.Div(
        style={
            'padding-top': '20px',
            'padding-bottom': '20px',
            'padding-right': '20px',
            'padding-left': '20px',
        },
        children=[
            html.H5("Les plus anciennes productions"),
            html.Table(
                style={
                    'border-style':'solid',
                    'border-width':'1px',
                    'padding-top': '20px',
                    'padding-bottom': '20px',
                    'padding-right': '20px',
                    'padding-left': '20px',
                    #'display': 'flex',
                    #'alignItems': 'center',
                    #'justifyContent': 'center',
                    #'flexDirection' : 'column',
                },
                children=[
                    html.Thead(
                        html.Tr(
                            children=[
                                html.Th('Titre'), 
                                html.Th('Année'), 
                                html.Th('Langue')
                            ]
                        )
                    ),
                    html.Tbody(
                        id = 'table-body-latest',
                        children=table_body_latest,
                    )
                ],
            ),
        ],  
    )
    
    table = html.Table(
        children=[
            html.Tr(
                children=[
                    html.Td(
                        children=[table_top_n],
                        rowSpan='2',
                        colSpan='2',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    ),
                    html.Td(
                        children=[table_latest],
                        rowSpan='1',
                        colSpan='1',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    )
                ]
            ),
            html.Tr(
                children=[
                    html.Td(
                        children=[table_oldest],
                        rowSpan='1',
                        colSpan='1',
                        #style={'border-style':'solid','border-width':'1px'}, 
                    )
                ]
            )
        ]
    )
                     
    return table

In [ ]:
my_df = load_data_for_treemap()
my_df = clean_year_column(my_df)
my_df = clean_genre_column(my_df)
my_df = clean_title_column(my_df)
#my_df = clean_language_column(my_df)
my_df = add_constant_col(my_df, "planete", "Terre")
year_min, year_max = get_min_and_max_year(df=my_df)
genres = get_list_of_genres(df=my_df)

treemap_fig = init_treemap_fig(df=my_df, year_min=year_min, year_max=year_max)

treemap_title = create_treemap_title(
    year_min_displayed=year_min, 
    year_max_displayed=year_max, 
    genre_displayed="All"
)
treemap_graph = create_treemap_graph(treemap_fig=treemap_fig)
treemap_ddm = create_treemap_dropdown_menu(genres=genres)
treemap_rs = create_treemap_range_slider(year_min=year_min, year_max=year_max)

top_n = 5
table_title = create_table_title(
    year_min_displayed=year_min, 
    year_max_displayed=year_max, 
    genre_displayed="All",
    top_n=top_n
)
table_slider = create_table_slider(initial_value=top_n)
table_body = create_table_body(
        df=my_df,
        genre="All",
        from_year=year_min, 
        to_year=year_max, 
        top_n=top_n
    )

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Projet INF8808 - Cinemathèque québécoise"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    create_treemap(
        treemap_title=treemap_title,
        treemap_rs=treemap_rs,
        treemap_ddm=treemap_ddm,
        treemap_graph=treemap_graph
    ),
    create_table(
        table_title=table_title,
        table_body=table_body,
        table_slider=table_slider
   )
])
@app.callback([Output('treemap-title', 'children'),
               Output('treemap-graph', 'figure'),
               Output('table-title', 'children'),
               Output('table-body', 'children')],
              [Input('treemap-dropdown-menu', 'value'),
               Input('treemap-range-slider', 'value'),
               Input('table-slider', 'value')])
def update_treemap(ddm_value, rs_value, s_value):     
    treemap_title, treemap_fig = callback_treemap(
        df=my_df,
        slider_min_year=rs_value[0], 
        slider_max_year=rs_value[1], 
        ddm_genre=ddm_value
    )
    
    table_title, table_body = callback_table(
        df=my_df,
        slider_min_year=rs_value[0], 
        slider_max_year=rs_value[1], 
        ddm_genre=ddm_value,
        top_n=s_value[0] 
    )
       
    return treemap_title, treemap_fig, table_title, table_body

#app.run_server(mode='inline', port=8010)
app.run_server(mode='jupyterlab', port = 8020, dev_tools_ui=True, #debug=True, 
               dev_tools_hot_reload =True, threaded=True)

In [69]:
my_df = load_data_for_treemap()
my_df = clean_year_column(my_df)
my_df = clean_genre_column(my_df)
my_df = clean_title_column(my_df)
my_df = clean_language_column(my_df)
my_df = drop_duplicated_movies(df=my_df)
my_df = add_constant_col(my_df, "planete", "Terre")
year_min, year_max = get_min_and_max_year(df=my_df)
genres = get_list_of_genres(df=my_df)

treemap_fig = init_treemap_fig(df=my_df, year_min=year_min, year_max=year_max)

treemap_title = create_treemap_title(
    year_min_displayed=year_min, 
    year_max_displayed=year_max, 
    genre_displayed="All"
)
treemap_graph = create_treemap_graph(treemap_fig=treemap_fig)
treemap_ddm = create_treemap_dropdown_menu(genres=genres)
treemap_rs = create_treemap_range_slider(year_min=year_min, year_max=year_max)

top_n = 5
table_title = create_table_title(
    year_min_displayed=year_min, 
    year_max_displayed=year_max, 
    genre_displayed="All",
    top_n=top_n
)
table_slider = create_table_slider(initial_value=top_n)
table_body_top_n, table_body_oldest, table_body_latest  = create_table_body(
        df=my_df,
        genre="All",
        from_year=year_min, 
        to_year=year_max, 
        top_n=top_n
    )

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
nom_de_la_page = "Projet INF8808 - Cinemathèque québécoise"
app = JupyterDash(nom_de_la_page, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    create_treemap(
        treemap_title=treemap_title,
        treemap_rs=treemap_rs,
        treemap_ddm=treemap_ddm,
        treemap_graph=treemap_graph
    ),
    create_table(
        table_title=table_title,
        table_body_top_n=table_body_top_n,
        table_body_oldest=table_body_oldest,
        table_body_latest=table_body_latest,
        table_slider=table_slider
   )
])
@app.callback([Output('treemap-title', 'children'),
               Output('treemap-graph', 'figure'),
               Output('table-title', 'children'),
               Output('table-body-top-n', 'children'),
               Output('table-body-oldest', 'children'),
               Output('table-body-latest', 'children')],
              [Input('treemap-dropdown-menu', 'value'),
               Input('treemap-range-slider', 'value'),
               Input('table-slider', 'value')])
def update_treemap(ddm_value, rs_value, s_value):     
    treemap_title, treemap_fig = callback_treemap(
        df=my_df,
        slider_min_year=rs_value[0], 
        slider_max_year=rs_value[1], 
        ddm_genre=ddm_value
    )
    
    table_title, table_body_top_n, table_body_oldest, table_body_latest = callback_table(
        df=my_df,
        slider_min_year=rs_value[0], 
        slider_max_year=rs_value[1], 
        ddm_genre=ddm_value,
        top_n=s_value[0] 
    )
       
    return treemap_title, treemap_fig, table_title, table_body_top_n, table_body_oldest, table_body_latest 

#app.run_server(mode='inline', port=8010)
app.run_server(port = 8020, dev_tools_ui=True, #debug=True, 
               dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:8020/


### Extra tools

In [40]:
fig.show()

In [ ]:
fig.full_figure_for_development(warn=False).show('json')